# Taylor Polynomial Approximations

Adam Rumpf

Created 4/17/21

Based on a <a href="https://github.com/adam-rumpf/mathematica-class-demonstrations#taylor-and-fourier-series-approximations" target="_blank">Mathematica class demonstration</a>.

See a standalone version of the main Taylor polynomial widget (shown [below](#Taylor-Series)) [here](./taylor-series-standalone.ipynb).

[Main Project Page](.././index.ipynb)

In [1]:
# Initialization code

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

# Define parameters
INF = 100 # out-of-bounds value
LIM = 10 # axis bounds

# Define functions and derivatives
def d_sin(x, n=0):
    """n-th derivative of sin(x)."""
    
    r = n % 4
    if r == 0:
        return np.sin(x)
    elif r == 1:
        return np.cos(x)
    elif r == 2:
        return -np.sin(x)
    elif r == 3:
        return -np.cos(x)

def d_cos(x, n=0):
    """n-th derivative of cos(x)."""
    
    return d_sin(x, n=n+1)

def d_exp(x, n=0):
    """n-th derivative of exp(x)."""
    
    return np.exp(x)

def d_log(x, n=0):
    """n-th derivative of log(x+1)."""
    
    if n == 0:
        return np.log(x+1)
    else:
        return (np.math.factorial(n-1) * (-1)**(n+1)) / (x+1)**n

def d_bell(x, n=0):
    """n-th derivative of a bell curve."""
    
    out = np.exp(-(x**2)/5) * 5**(-(n-1))
    if n == 0:
        return out
    elif n == 1 or n == 2:
        out *= -2
        if n == 1:
            out *= x
        elif n == 2:
            out *= 5 - 2*(x**2)
    elif n == 3 or n == 4:
        out *= 4
        if n == 3:
            out *= (15 - 2*(x**2))*x
        elif n == 4:
            out *= 75 - 60*(x**2) + 4*(x**4)
    elif n == 5 or n == 6:
        out *= -8
        if n == 5:
            out *= (375 - 100*(x**2) + 4*(x**4))*x
        elif n == 6:
            out *= 1875 - 2250*(x**2) + 300*(x**4) - 8*(x**6)
    elif n == 7 or n == 8:
        out *= 16
        if n == 7:
            out *= (13125 - 5250*(x**2) + 420*(x**4) - 8*(x**6))*x
        elif n == 8:
            out *= 65625 - 105000*(x**2) + 21000*(x**4) - 1120*(x**6) + 16*(x**8)
    elif n == 9 or n == 10:
        out *= -32
        if n == 9:
            out *= (590625 - 315000*(x**2) + 37800*(x**4) + 1440*(x**6) + 16*(x**8))*x
        elif n == 10:
            out *= 2953125 - 5906250*(x**2) + 1575000*(x**4) - 126000*(x**6) + 3600*(x**8) - 32*(x**10)
    return out

def d_poly(x, n=0):
    """n-th derivative of a polynomial function."""
    
    if n == 0:
        return 0.35 + x*(0.16 + x*(-0.1875 + x*(0.005 + x*0.0025)))
    elif n == 1:
        return 0.16 + x*(-0.375 + x*(0.015 + x*0.01))
    elif n == 2:
        return -0.375 + x*(0.03 + x*0.03)
    elif n == 3:
        return 0.03 + x*0.06
    elif n == 4:
        return 0.06 + x*0.0
    else:
        return x*0.0

def d_ratio(x, n=0):
    """n-th derivative of 1/(x+1)."""
    
    return ((-1)**n)/((x+1)**(n+1))

# Define a dictionary of function definitions
func = {}
func["sine"] = d_sin
func["cosine"] = d_cos
func["exponential"] = d_exp
func["logarithm"] = d_log
func["bell curve"] = d_bell
func["polynomial"] = d_poly
func["rational"] = d_ratio

# Define a dictionary of function name strings
func_name = {}
func_name["sine"] = "$\sin x$"
func_name["cosine"] = "$\cos x$"
func_name["exponential"] = "$e^x$"
func_name["logarithm"] = "$\log(x+1)$"
func_name["bell curve"] = "$5e^{-x^2/5}$"
func_name["polynomial"] = "$0.35 + 0.16x - 0.1875x^2 + 0.005x^3 + 0.0025x^4$"
func_name["rational"] = "$1/(x+1)$"

# Define Taylor polynomial
def taylor(x, fname, a, n):
    """Taylor polynomial for a given function.
    
    Positional arguments:
    x - input value
    fname - key from 'func' dictionary
    a - center
    n - polynomial degree
    """
    
    out = 0.0 # output value
    
    # Add terms of Taylor polynomial
    for i in range(n+1):
        out += (func[fname](a, n=i) / np.math.factorial(i)) * (x-a)**i
    
    return out

xbase = np.linspace(-LIM, LIM, 101) # base x-values

## Motivation for Polynomial Approximation

In computational mathematics it's often useful to approximate a complicated function using a simpler one. This could be because the real function we're interested in is computationally expensive to evaluate, or because it leads to overly-complicated analytical results, or because it's actually unknown (as is the case in solving differential equations). However, as long as we have a bit of information about our function for one particular input, and as long as we only need to move a small distance away from that single known input, we may be able to reasonably approximate its value within a small range.

Specifically, suppose we have a function $f(x)$ that we would like to approximate around some input value $a$. Our goal is to find a "simple" function $p(x)$ that is "approximately equal" to $f(x)$ for values of $x$ "near" $a$. That is,

\begin{align*}
\text{Find the polynomial $p(x)$ that is as close as possible to $f(x)$ near $a$.}
\end{align*}

There are lots of reasonable choices for what kind of function we could choose $p$ to be, but we happen to get some particularly nice results when we look at the case of $p$ being a polynomial.

## Building Up a Polynomial Approximation

Intuitively our goal is to define a polynomial $p(x)$ that is as close as possible to $f(x)$ near $a$. How exactly we do that depends on the degree of the polynomial, so let's go through the process of building up a sequence of increasingly-complicated degree-$n$ approximations $p_n$ to see whether we find any patterns that can be generalized.

### Constant Approximation

What degree-zero polynomial $p_0(x) = c_0$ does the best job of approximating $f(x)$ near $a$?

A degree-zero polynomial is constant, so our only real job is to choose which constant value $c_0$ we want $p_0(x)$ to have. Since we've only been told to care about the input value $a$, the only thing we can really do is to guarantee that $p_0$ approximates $f$ exactly when $x=a$ by setting $c_0 = f(a)$. Then we get

\begin{align*}
p_0(x) &= f(a)
\end{align*}

In [2]:
# Set up plot
fig1, ax1 = plt.subplots()

# Draw plot lines
@widgets.interact(a=(-LIM, LIM, 0.05))
def update1(a=0.0):
    
    global ax
    fname = "polynomial"
    n=0
    a0 = a
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax1.clear()
    ax1.set_xlim([-LIM, LIM])
    ax1.set_ylim([-LIM, LIM])
    ax1.set_aspect(1)
    ax1.grid(False)
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax1.plot(x, func[fname](x), color="C0")
    ax1.plot(xbase, taylor(xbase, fname, a0, n), color="C1")
    ax1.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='a', max=10.0, min=-10.0, step=0.05), Output()), _dom…

The above widget shows a constant approximation of a function. Click and drag the slider bar to adjust the center of the approximation, $a$.

This is an okay approximation as long as the function value doesn't change very quickly around $a$, but otherwise $f(x)$ quickly diverges from $p_0(x)$ as $x$ moves further from $a$. However, we can try to fix that by using a more complicated polynomial approximation that takes this rate of change into account.

### Linear Approximation

What degree-one polynomial $p_1(x) = c_0 + c_1 x$ does the best job of approximating $f(x)$ near $a$?

A degree-one polynomial is linear, which gives us one additional coefficient to manipulate: Not only can we control the value of $p_1(x)$ and $a$, we can also control its _slope_. We certainly want to keep the property that $p_1(a) = f(a)$, since that means that $p_1(x)$ is as close as possible to $f(x)$ when $x$ is exactly equal to $a$, but what slope should we choose to make $p_1(x)$ as close to $f(x)$ as possible as we move away from $a$?

In [3]:
# Set up plot
fig2, ax2 = plt.subplots()

# Draw plot lines
@widgets.interact(slope=(-4.0, 4.0, 0.05))
def update2(slope=0.5):
    
    global ax
    fname = "polynomial"
    a0 = -1.0
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax2.clear()
    ax2.set_xlim([-LIM, LIM])
    ax2.set_ylim([-LIM, LIM])
    ax2.set_aspect(1)
    ax2.grid(False)
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    ax2.plot(x, func[fname](x), color="C0")
    ax2.plot(xbase, func[fname](a0) + slope*(xbase - a0), color="C1")
    ax2.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, description='slope', max=4.0, min=-4.0, step=0.05), Output()), _d…

The above widget shows a linear approximation of a function. Click and drag the slider bar to adjust the slope.

Intuitively we would like to choose the slope of $p_1(x)$ to match the slope of $f(x)$ as closely as possible near $a$, which we can do by setting the slope of $p_1(x)$ to equal the derivative of $f$ at $a$, or $f'(a)$.

Since we know that $p_1(x)$ should be a line passing through point $(a,f(a))$ with slope $f'(a)$, we can apply the point-slope formula for a line to arrive at the formula

\begin{align*}
p_1(x) &= f(a) + f'(a) \, (x-a)
\end{align*}

(Alternatively, we could arrive at this by beginning with the special case of $a=0$, in which case $(a,f(a)) = (0,f(0))$ is the same as the $y$-intercept and we can apply the slope-intercept formula to get $p_1(x) = f(a) + f'(a) \, x$. To generalize this to any $a$ we simply need to horizontally shift the $a=0$ graph by $a$ units by replacing $x$ with $x-a$, which gives the general linearization above.)

This is known as the _linear approximation_ of $f$ at $a$, and it appears as an essential part of many advanced numerical algorithms (such as the <a href="https://en.wikipedia.org/wiki/Newton%27s_method" target="_blank">Newton-Raphson method</a> for root finding and <a href="https://en.wikipedia.org/wiki/Euler_method" target="_blank">Euler's method</a> for numerical differential equation solution). Geometrically it is equivalent to finding the tangent line of $f$ at $a$.

In [4]:
# Set up plot
fig3, ax3 = plt.subplots()

# Draw plot lines
@widgets.interact(a=(-LIM, LIM, 0.05))
def update3(a=0.0):
    
    global ax
    fname = "polynomial"
    a0 = a
    n = 1
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax3.clear()
    ax3.set_xlim([-LIM, LIM])
    ax3.set_ylim([-LIM, LIM])
    ax3.set_aspect(1)
    ax3.grid(False)
    ax3.set_xlabel("x")
    ax3.set_ylabel("y")
    ax3.plot(x, func[fname](x), color="C0")
    ax3.plot(xbase, taylor(xbase, fname, a0, n), color="C1")
    ax3.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='a', max=10.0, min=-10.0, step=0.05), Output()), _dom…

The above widget shows a linear approximation of a function, with its function value and slope automatically chosen to match those of the function at the center. Click and drag the slider bar to adjust the center of the approximation, $a$.

This is an okay approximation as long as the _slope_ of the function doesn't change very quickly around $a$, but if $f(x)$ curves very sharply then the linear approximation does not remain reasonable as $x$ moves away from $a$. So our next goal is to take this curvature into account.

### Quadratic Approximation

What degree-two polynomial $p_2(x) = c_0 + c_1 x + c_2 x^2$ does the best job of approximating $f(x)$ near $a$?

Since the previous two approximations consisted of ensuring that our polynomial would have the same function value as $f$ at $a$ (i.e. $p(a) = f(a)$) and the same slope as $f$ at $a$ (i.e. $p'(a) = f'(a)$), a reasonable next goal might be to ensure the correct _curvature_ as $f$ at $a$ (i.e. $p''(a) = f''(a)$). Looking at the previous two approximations, we might notice that the degree-zero approximation appears within the degree-one approximation,

\begin{align*}
p_0(x) &= f(a) \\
p_1(x) &= f(a) + f'(a) \, (x-a)
\end{align*}

This might cause us to wonder whether the solution could be as simple as just adding a third term to the polynomial of the form $c_2 (x-a)^2$, similar to how the previous approximation consisted of adding a second term of the form $c_1 (x-a)$. If such a thing were possible, we would want to choose $c_2$ in order to ensure that we maintain our previous two properties (that $p_2(a) = f(a)$ and $p'_2(a) = f'(a)$), but now also that $p''_2(a) = f''(a)$.

In [5]:
# Set up plot
fig4, ax4 = plt.subplots()

# Draw plot lines
@widgets.interact(curvature=(-2.0, 2.0, 0.05))
def update4(curvature=-0.1):
    
    global ax
    fname = "polynomial"
    a0 = -1.0
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax4.clear()
    ax4.set_xlim([-LIM, LIM])
    ax4.set_ylim([-LIM, LIM])
    ax4.set_aspect(1)
    ax4.grid(False)
    ax4.set_xlabel("x")
    ax4.set_ylabel("y")
    ax4.plot(x, func[fname](x), color="C0")
    ax4.plot(xbase, taylor(x, fname, a0, 1) + curvature*(x - a0)**2, color="C1")
    ax4.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-0.1, description='curvature', max=2.0, min=-2.0, step=0.05), Output()…

The above widget shows a quadratic approximation of a function. Click and drag the slider bar to adjust the curvature.

This goal helps to explain why adding a third term of the form $c_2 (x-a)^2$ will help to give us the desired properties. The function value of $p_2$ at $a$ is

\begin{align*}
p_2(x) &= f(a) + f'(a) \, (x-a) + c_2 \, (x-a)^2 \\
\implies p_2(a) &= f(a) + f'(a) \, (a-a) + c_2 \, (a-a)^2 \\
&= f(a)
\end{align*}

which is the same function value as $f$ at $a$, as desired. The factor of $(x-a)$ in all non-constant terms vanishes when $x=a$, leaving only the constant term $f(a)$. Therefore we see that the constant term $f(a)$ is solely responsible for the function value of $p_2$ at $a$.

Likewise, the first derivative of $p_2$ at $a$ is

\begin{align*}
p'_2(x) &= f'(a) + 2c_2 \, (x-a) \\
\implies p'_2(a) &= f'(a) + 2c_2 \, (a-a) \\
&= f'(a)
\end{align*}

which is the same first derivative as $f$ at $a$, also as desired. The constant term $f(a)$ disappears when evaluating the first derivative and the quadratic term retains a factor of $(x-a)$ which vanishes when $x=a$, this time leaving only the coefficient of the linear term $f'(a)$. Therefore the linear term $f'(a)$ is solely responsible for the first derivative of $p_2$ at $a$.

Now to choose the quadratic coefficient $c_2$ to ensure that $p_2$ has the same second derivative as $f$ at $a$. Evaluating the second derivative, we are left with simply

\begin{align*}
p''_2(x) &= 2c_2
\end{align*}

so in order to have $p''_2(a) = f''(a)$ we can simply choose $c_2 = \frac{1}{2} f''(a)$. In this case the quadratic term $\frac{1}{2} f''(a)$ is solely responsible for the second derivative of $p_2$ at $a$, but we need to be careful to apply a factor of $\frac{1}{2}$ to account for the power rule being applied to $(x-a)^2$ in evaluating the second derivative of $p_2$. The full quadratic approximation is then

\begin{align*}
p_2(x) = f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2
\end{align*}

In [6]:
# Set up plot
fig5, ax5 = plt.subplots()

# Draw plot lines
@widgets.interact(a=(-LIM, LIM, 0.05))
def update5(a=0.0):
    
    global ax
    fname = "polynomial"
    a0 = a
    n = 2
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax5.clear()
    ax5.set_xlim([-LIM, LIM])
    ax5.set_ylim([-LIM, LIM])
    ax5.set_aspect(1)
    ax5.grid(False)
    ax5.set_xlabel("x")
    ax5.set_ylabel("y")
    ax5.plot(x, func[fname](x), color="C0")
    ax5.plot(xbase, taylor(xbase, fname, a0, n), color="C1")
    ax5.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='a', max=10.0, min=-10.0, step=0.05), Output()), _dom…

The above widget shows a quadratic approximation of a function, with its function value, slope, and now curvature automatically chosen to match those of the function at the center. Click and drag the slider bar to adjust the center of the approximation, $a$.

In developing this most recent approximation we came upon an extremely useful property: If we set up our polynomial approximation in the right way, we can ensure that every term except for one vanishes when we evaluate a derivative of the polynomial at $a$, which then allows us to select each coefficient of the polynomial to independently define one of the derivatives of the polynomial at $a$. This process can be easily generalized to obtain a degree-$n$ polynomial whose first $n$ derivatives ($0$ through $n-1$) all match those of $f$ at $a$, since each new derivative can be matched by simply adding one new term to the polynomial without needing to change any of the previous ones.

However, we also saw in the previous example that some of these coefficients require an additional factor, so it might be helpful to go through the next couple of approximations by hand to work out the general form.

### Cubic Approximations

What degree-three polynomial $p_3(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3$ does the best job of approximating $f(x)$ near $a$?

Picking up from the previous example, we might imagine that the next approximation has the form

\begin{align*}
p_3(x) &= f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2 + c_3 \, (x-a)^3
\end{align*}

From the previous example we know that this polynomial satisfies the properties $p_3(a) = f(a)$, $p'_3(a) = f'(a)$, and $p''_3(a) = f''(a)$, since the lone new term includes a factor of $(x-a)$ which vanishes at $x=a$ in the zeroth, first, and second derivatives of $p_3$. By the time we evaluate the third derivative, all previous terms have vanished, leaving only

\begin{align*}
p'''_3(x) &= 3 \cdot 2 \cdot c_3
\end{align*}

with the factors of $3$ and $2$ left over from the first two applications of the power rule to $(x-a)^3$. In order to have $p'''_3(a) = f'''(a)$ we should then choose $c_3 = \frac{1}{6} f'''(a)$, for a cubic approximation of

\begin{align*}
p_3(x) = f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2 + \frac{1}{6} f'''(a) \, (x-a)^3
\end{align*}

In [7]:
# Set up plot
fig6, ax6 = plt.subplots()

# Draw plot lines
@widgets.interact(a=(-LIM, LIM, 0.05))
def update6(a=0.0):
    
    global ax
    fname = "polynomial"
    a0 = a
    n = 3
    
    # Generate function values
    x = np.linspace(-LIM, LIM, 101)
    
    # Redraw plot
    ax6.clear()
    ax6.set_xlim([-LIM, LIM])
    ax6.set_ylim([-LIM, LIM])
    ax6.set_aspect(1)
    ax6.grid(False)
    ax6.set_xlabel("x")
    ax6.set_ylabel("y")
    ax6.plot(x, func[fname](x), color="C0")
    ax6.plot(xbase, taylor(xbase, fname, a0, n), color="C1")
    ax6.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='a', max=10.0, min=-10.0, step=0.05), Output()), _dom…

### Quartic Approximations

What degree-four polynomial $p_4(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3 + c_4 x^4$ does the best job of approximating $f(x)$ near $a$?

Going through a process similar to the above cases, we find that the new term must be $\frac{1}{24} f^{(4)}(a) \, (x-a)^4$, with the factor of $\frac{1}{24}$ to compensate for applying the power rule to $(x-a)^4$ four times. This pattern can be seen more easily if we write $\frac{1}{4!}$ in place of $\frac{1}{24}$, to show that the division by $4! = 4 \cdot 3 \cdot 2 \cdot 1$ compensates for the four applications of the power rule. Then the quartic approximation is

\begin{align*}
p_4(x) &= f(a) + f'(a) \, (x-a) + \frac{f''(a)}{2} (x-a)^2 + \frac{f'''(a)}{3!} (x-a)^3 + \frac{f^{(4)}(a)}{4!} (x-a)^4
\end{align*}

### Generalizing Polynomial Approximations

From our previous work we can see that adding more, higher-degree terms of a certain form allows us to match more, higher-order derivatives of $f$ at $a$. Intuitively this allows the polynomial approximation to match $f$ more closely over a larger range of inputs since it allows the polynomial to compensate for changes in the lower-order derivatives (matching the slope allows us to account for a rapidly-changing function value, matching curvature allows us to account for a rapidly-changing slope, etc.). The general form of the degree-$n$ term is

\begin{align*}
\frac{f^{(n)}(a)}{n!} (x-a)^n
\end{align*}

with a factor of $\frac{1}{n!}$ to compensate for $n$ applications of the power rule to $(x-a)^n$. This term is solely responsible for defining the $n$th derivative of $p_n(x)$, and ensures that $p_n^{(n)}(a) = f^{(a)}(a)$.

Written using summation notation, the degree-$n$ polynomial approximation of $f$ at $a$ is

\begin{align*}
p_n(x) = \sum_{m=0}^n \frac{f^{(m)}(a)}{m!} (x-a)^m
\end{align*}

Polynomial approximations of this form are called <a href="https://en.wikipedia.org/wiki/Taylor_series" target="_blank">Taylor polynomials</a>, and are used extensively throughout numerical analysis. In addition to the approximation applications discussed above, Taylor polynomials are often used in deriving error bounds and convergence criteria for numerical methods, since there are good theoertical bounds that can be placed on how close $p_n(x)$ is to $f(x)$.

## Taylor Series

While we set out with the goal of finding a "simple" function (a polynomial) that would be "close" to $f$ "near" $a$, one of the most surprising results about Taylor polynomials is that, for many functions, this sequence of polynomial approximations can actually be extended to include infinitely many terms, at which point they cease to simply _approximate_ $f$ near $a$ and actually become exactly _equal_ to $f$ over the entire real line. This allows us to write $f(x)$ as an equivalent infinite series

\begin{align*}
f(x) = \sum_{n=0}^\infty \frac{f^{(n)}(a)}{n!} (x-a)^n
\end{align*}

This is called the _Taylor series_ of $f$ at $a$, and it is an important tool used extensively throughout mathematical analysis since the resulting series is often easier to use or analyze than the original function.

The widget below allows the user to select from a small handful of simple functions to see their Taylor polynomial approximations for various centers $a$ and degrees $n$. Note that some functions are extremely close to their Taylor polynomial approximations for a large range of values around $a$, while others don't seem able to be approximated well over a large range.

In [8]:
# Set up plot
fig, ax = plt.subplots()
xbase = np.linspace(-LIM, LIM, 101) # base x-values

# Draw plot lines
@widgets.interact(fname=func.keys(), a=(-LIM, LIM, 0.05), n=(0, 10, 1))
def update_final(fname="sine", a=0.0, n=1):
    
    global ax
    a0 = a
    
    # Generate function values
    if fname == "logarithm":
        x = np.linspace(-0.99, LIM, 101)
        a0 = max(a0, -0.9)
    elif fname == "rational":
        x = np.linspace(-LIM, LIM, 100)
        if a0 == -1.0:
            a0 += 0.05
    else:
        x = np.linspace(-LIM, LIM, 101)
    y = np.zeros_like(x)
    
    # Redraw plot
    ax.clear()
    ax.set_xlim([-LIM, LIM])
    ax.set_ylim([-LIM, LIM])
    ax.set_aspect(1)
    plt.title(func_name[fname])
    ax.grid(False)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.plot(x, func[fname](x), color="C0")
    y = taylor(xbase, fname, a0, n)
    ax.plot(xbase, y, color="C1")
    ax.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)
    if fname in {"logarithm", "rational"}:
        ax.plot([-1, -1], [-INF, INF], color="white")
        ax.plot([-1, -1], [-INF, INF], color="black", linestyle="dashed")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='fname', options=('sine', 'cosine', 'exponential', 'logarithm', 'be…